# Día 3: Tipologías de Modelos y Detección de Anomalías en Series Temporales

En este notebook cubriremos los siguientes puntos:
1. **Generación (o carga) de datos** para trabajar.
2. **Resumen de tipologías de modelos**:
   - Modelos estadísticos (ARIMA).
   - Modelo supervisado simple.
   - Modelo no supervisado (Isolation Forest).
3. **Detección de anomalías**:
   - Método estadístico simple (z-score).
   - Isolation Forest.
4. **Comparación de resultados**.
5. **Tarea** para profundizar.

**Objetivo**: Familiarizarnos con distintas categorías de modelos y aprender métodos básicos de detección de anomalías en datos temporales.


In [ ]:
# Sección 0: Importaciones y Configuración

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos estadísticos
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

# Para modelo supervisado
from sklearn.linear_model import LinearRegression

# Para detección de anomalías no supervisada
from sklearn.ensemble import IsolationForest

# Para métricas y z-score
from scipy.stats import zscore

%matplotlib inline
sns.set_style("whitegrid")

print("Entorno de trabajo configurado.")


## Sección 1: Generación (o Carga) de Datos

Para esta práctica, generaremos de forma **sintética** una serie temporal con:
- Tendencia creciente.
- Estacionalidad mensual (aprox. 30 días).
- Ruido aleatorio.
- **Anomalías inyectadas** en ciertos puntos, para ilustrar la detección de outliers.

Si deseas usar un dataset propio o real, reemplaza la siguiente celda por una lectura de datos, por ejemplo:

```python
df = pd.read_csv("mi_dataset.csv", parse_dates=["fecha"], index_col="fecha")


In [ ]:
# Sección 1: Creación de datos sintéticos con anomalías

np.random.seed(42)  # Semilla para reproducibilidad

date_range = pd.date_range(start='2025-01-01', end='2025-12-31', freq='D')
n = len(date_range)

# Tendencia lineal
trend = np.linspace(50, 150, n)

# Estacionalidad (aprox 30 días)
seasonality = 10 * np.sin(2 * np.pi * np.arange(n) / 30)

# Ruido normal
noise = np.random.normal(loc=0, scale=5, size=n)

# Generamos la serie base
values = trend + seasonality + noise

# Inyectamos algunas anomalías (picos altos o muy bajos)
anomaly_indices = np.random.choice(n, size=5, replace=False)  # 5 anomalías al azar
values[anomaly_indices] += np.random.choice([30, -30], size=5)  # Añadimos un salto

# Creamos DataFrame
df = pd.DataFrame({'consumo': values}, index=date_range)

df.head()


Observa que `df` tiene dos columnas:
- **Index**: la fecha.
- **consumo**: la serie con tendencia, estacionalidad, ruido y algunos valores anómalos.

---
## Sección 2: Visualización Inicial

Antes de entrar en los modelos, revisamos la serie para ver su apariencia general y, si es posible, detectar "a simple vista" las anomalías.


In [ ]:
# Sección 2: Visualización de la serie
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['consumo'], label='Consumo', alpha=0.8)
plt.title('Serie Temporal con posibles anomalías')
plt.xlabel('Fecha')
plt.ylabel('Consumo')
plt.legend()
plt.show()


Si tenemos suerte, algunas anomalías (puntos muy altos o muy bajos) se verán a simple vista.

---

## Sección 3: Modelos Estadísticos Clásicos (ARIMA)

### 3.1. Breve explicación
- **ARIMA(p, d, q)** modela la serie como una combinación de partes autorregresivas (AR), diferencias (I) y medias móviles (MA).
- Para este ejemplo, **no** realizaremos un tuning completo de p, d, q. Solo mostraremos cómo ajustar un ARIMA simple con `statsmodels`.

### 3.2. Ejemplo: Ajuste ARIMA a nuestra serie


In [ ]:
# Sección 3: Ajuste simple de un modelo ARIMA
# Para un ejemplo rápido, seleccionamos manualmente p, d, q:
p, d, q = 2, 1, 2

# Generamos la serie en un formato un poco más "clásico" para ARIMA (simple array)
serie = df['consumo']

# Ajustamos el modelo
model = ARIMA(serie, order=(p, d, q))
results = model.fit()

print(results.summary())


**Interpretación**:
- El output de `.summary()` nos da parámetros (AR, MA) y medidas de ajuste (AIC, BIC).
- Podríamos intentar diferentes órdenes (p,d,q) o usar funciones como `pmdarima` para auto-ARIMA, pero eso excede el enfoque de esta práctica.

### 3.3. Pronóstico con el ARIMA ajustado


In [ ]:
# Pronosticamos algunos días del futuro (ej. 15 días)
pred_steps = 15
forecast = results.forecast(steps=pred_steps)

# Construimos un DataFrame para visualización
forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=pred_steps, freq='D')
forecast_df = pd.DataFrame({'forecast': forecast.values}, index=forecast_index)

# Graficamos
plt.figure(figsize=(12,5))
plt.plot(df.index, df['consumo'], label='Histórico')
plt.plot(forecast_df.index, forecast_df['forecast'], label='Pronóstico ARIMA', color='red')
plt.legend()
plt.title('Pronóstico con ARIMA')
plt.show()


Vemos el resultado del modelo ARIMA (muy básico). No esperes una gran precisión si la serie contiene anomalías o si la estacionalidad no se manejó con SARIMA.  
Esto ilustra el **modelo estadístico clásico**.

---

## Sección 4: Modelo Supervisado Sencillo (Regresión Lineal)

### 4.1. Idea General
- Podemos tratar la predicción de la serie como un problema de **regresión supervisada**.
- Generamos features a partir de la propia serie (lags) y quizás alguna variable exógena (en este ejemplo, no la usaremos).

### 4.2. Construcción de Features


In [ ]:
# Sección 4: Modelo supervisado sencillo

df_supervised = df.copy()

# Creamos características: consumo (t-1), consumo (t-2)
df_supervised['consumo_lag1'] = df_supervised['consumo'].shift(1)
df_supervised['consumo_lag2'] = df_supervised['consumo'].shift(2)

# Eliminamos filas iniciales con NaN por shift
df_supervised.dropna(inplace=True)

# Definimos X e y
X = df_supervised[['consumo_lag1', 'consumo_lag2']]
y = df_supervised['consumo']

# Separamos un conjunto de entrenamiento y test (por ejemplo, 80% para entrenar)
split_index = int(len(df_supervised)*0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Entrenamos modelo de regresión lineal
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Coeficientes:", lr.coef_)
print("Intercepto:", lr.intercept_)


### 4.3. Predicción y visualización

In [ ]:
y_pred = lr.predict(X_test)

# Armamos DataFrame para graficar
df_pred = pd.DataFrame({'real': y_test, 'pred': y_pred}, index=y_test.index)

plt.figure(figsize=(12,5))
plt.plot(df_pred.index, df_pred['real'], label='Real', alpha=0.7)
plt.plot(df_pred.index, df_pred['pred'], label='Predicción Reg. Lineal', alpha=0.7)
plt.title('Predicción con Modelo Supervisado (Reg. Lineal)')
plt.legend()
plt.show()


**Comentario**:
- El modelo se basa únicamente en valores pasados (`lag1`, `lag2`); no tiene en cuenta estacionalidad a largo plazo ni anomalías explícitamente.
- Aun así, sirve para ilustrar cómo un **modelo supervisado** puede predecir la serie.

---

## Sección 5: Detección de Anomalías

Ahora que tenemos la serie (con algunos outliers inyectados), veremos dos enfoques:

1. **Método estadístico simple (z-score)**.  
2. **Isolation Forest** (no supervisado).

### 5.1. Método Estadístico (z-score)


In [ ]:
# Sección 5.1: Detección con z-score

df_anomalies = df.copy()
df_anomalies['z_score'] = zscore(df_anomalies['consumo'])  # scipy.stats.zscore

threshold = 3  # más allá de +/-3 se puede considerar outlier
df_anomalies['anomaly_z'] = df_anomalies['z_score'].apply(lambda x: 1 if abs(x) > threshold else 0)

# Visualizamos cuántos outliers detectamos
print("Número de anomalías detectadas (z-score):", df_anomalies['anomaly_z'].sum())

# Graficamos la serie, resaltando anomalías
plt.figure(figsize=(12,5))
plt.plot(df_anomalies.index, df_anomalies['consumo'], label='Consumo', alpha=0.7)

# Resaltamos en rojo los puntos outliers
plt.scatter(df_anomalies.index[df_anomalies['anomaly_z'] == 1],
            df_anomalies['consumo'][df_anomalies['anomaly_z'] == 1],
            color='red', label='Anomalía (z-score)', alpha=0.9)

plt.title('Detección de Anomalías con z-score')
plt.xlabel('Fecha')
plt.ylabel('Consumo')
plt.legend()
plt.show()


Observa que este método **asume** que la mayoría de los valores se distribuyen de forma aproximadamente normal.  
Si en tu serie hay estacionalidad o tendencia fuerte, es útil primero **centrar** la serie (por ejemplo, restarle la media móvil o usar los residuales).

### 5.2. Detección con Isolation Forest

Un método **no supervisado** que “aísla” puntos anómalos más rápidamente que puntos normales en un bosque de árboles aleatorios.


In [ ]:
# Sección 5.2: Detección con Isolation Forest

df_iforest = df.copy()

# Configuramos el modelo
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
# Contamination ~ ratio de anomalías esperado (1% en este ejemplo)

# Ajustamos y predecimos
iso_forest.fit(df_iforest[['consumo']])
df_iforest['anomaly_if'] = iso_forest.predict(df_iforest[['consumo']])
# Isolation Forest retorna -1 para anómalo, 1 para normal

# Convertimos a 1/0
df_iforest['anomaly_if'] = df_iforest['anomaly_if'].apply(lambda x: 1 if x == -1 else 0)

print("Número de anomalías detectadas (Isolation Forest):", df_iforest['anomaly_if'].sum())

# Visualizamos
plt.figure(figsize=(12,5))
plt.plot(df_iforest.index, df_iforest['consumo'], label='Consumo', alpha=0.7)

plt.scatter(df_iforest.index[df_iforest['anomaly_if'] == 1],
            df_iforest['consumo'][df_iforest['anomaly_if'] == 1],
            color='orange', label='Anomalía (IForest)', alpha=0.9)

plt.title('Detección de Anomalías con Isolation Forest')
plt.xlabel('Fecha')
plt.ylabel('Consumo')
plt.legend()
plt.show()


**Comentarios**:
- Con `contamination=0.01`, forzamos a que ~1% de los datos sean considerados anómalos. Podemos ajustar este hiperparámetro según el dominio del problema.
- A diferencia del z-score, Isolation Forest no asume nada sobre la distribución de los datos, pero requiere un volumen razonable de muestras.

---

## Sección 6: Comparación de Enfoques

Podemos comparar cuántas anomalías detecta cada método y si coinciden:


In [ ]:
df_compare = pd.DataFrame({
    'z_score': df_anomalies['anomaly_z'],
    'iforest': df_iforest['anomaly_if']
}, index=df.index)

df_compare['match'] = (df_compare['z_score'] == df_compare['iforest']).astype(int)

print("Coincidencias en la etiqueta de anomalía:", df_compare['match'].sum(), "de", len(df_compare))

Posiblemente, algunos puntos marcados como anomalía por z-score no lo sean para Isolation Forest, y viceversa. Esto demuestra cómo los **métodos** tienen distintos supuestos y sensibilidades.

---

## Sección 7: Tarea

1. **Probar distintos umbrales** de z-score (por ejemplo, 2.5, 3, 3.5) y ver cómo cambian las detecciones.
2. **Ajustar** el parámetro `contamination` de Isolation Forest (por ejemplo, 0.01, 0.02, 0.05) para ver el impacto.
3. (Opcional) **Implementar** un método de clustering (por ejemplo, K-Means) en ventanas de la serie para detectar outliers en la serie.
4. **Consultar** la documentación de `statsmodels` para un modelo SARIMA que maneje la estacionalidad más adecuadamente.

---

# Conclusión

En este notebook, hemos:
- Explorado **tipologías de modelos** (ARIMA, Regresión Lineal, Isolation Forest).
- Visto **detección de anomalías** con un método simple (z-score) y uno no supervisado (Isolation Forest).
- Aprendido la **importancia** de adaptar los parámetros y el preprocesamiento (estacionalidad, tendencia) antes de marcar puntos como anómalos.

¡Con esto concluye la parte práctica del Día 3!


Recomendaciones finales
Ajusta o profundiza según el nivel de la audiencia. Por ejemplo, si quieres enseñar SARIMA con statsmodels, muestra cómo se definen (p, d, q)(P, D, Q)m y cómo buscar los parámetros.
Para modelos supervisados más avanzados, podrías integrar RandomForestRegressor o XGBoost, pero lo esencial es entender la idea de “generar features” a partir de la serie.
En detección de anomalías, si tu dataset es multivariable, podrías alimentar Isolation Forest con varias columnas (ej. consumo, temperatura, festivo), no solo consumo.
Recuerda que la práctica (ensayo y error) es fundamental para ver cómo cambia la detección de anomalías ante diferentes hiperparámetros.